In [161]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re

In this notebook, we will try to collect all of the Authors listed on Project Gutemberg's website using the BeautifulSoup library.

In [52]:
authweb = "http://www.gutenberg.org/browse/authors/s"
r = requests.get(authweb).text
soup = bs(r, "html.parser")

In [53]:
print(soup.prettify()[:812])

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01//EN" "http://www.w3.org/TR/html4/strict.dtd">
<html lang="en">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="ebooks, ebook, books, book, free, online, audio" name="keywords"/>
  <meta content="33000+ free ebooks online" name="description"/>
  <meta content="public" name="classification"/>
  <meta content="text/css" http-equiv="Content-Style-Type"/>
  <script type="application/javascript">
   if (top != self) {
        top.location.replace ('http://www.gutenberg.org');
        alert ('Project Gutenberg is a FREE service with NO membership required. If you paid somebody else to get here, make them give you your money back!');
      }
  </script>
  <link href="/css/pg-002.css" rel="stylesheet" type="text/css"/>


BeautifulSoup has various types of elements. The most common to work with are the 'Tag' element, which may contain other nested Tags, and the 'NavigableString' type, which is text found in the HTML document

In [54]:
print(f"{soup.title}\t {type(soup.title)}")

<title>Browse By Author: S - Project Gutenberg</title>	 <class 'bs4.element.Tag'>


In [55]:
print(f"{soup.title.string}\t {type(soup.title.string)}")

Browse By Author: S - Project Gutenberg	 <class 'bs4.element.NavigableString'>


In [159]:
author = "Shakespeare, William"
#author = "Sacro Bosco"

# List of all h2 Tags
h2s = soup.find_all("h2")
for h2 in h2s:
    h2_link = h2.a
    # If a link is found, it is an author
    if h2_link is not None:
        author_match = h2_link.string
        if author in author_match:
            elements = h2.next_sibling.next_sibling.find_all(attrs = {"class":"pgdbetext"})
            regex = re.compile("( \(as [A-Za-z]+\)|\(|\))")
            titles, languages, links = list(), list(), list()
            for element in elements:
                titles.append(element.a.text)
                links.append(element.a.get("href"))
                languages.append(regex.sub("", element.contents[1].strip()))

In [166]:
pd.DataFrame({"title":titles, "language":languages, "link":links}).query("language == 'English'")

,language,link,title
1,English,/ebooks/2246,All's Well That Ends Well
2,English,/ebooks/1529,All's Well That Ends Well
3,English,/ebooks/1791,All's Well That Ends Well
4,English,/ebooks/1125,All's Well That Ends Well
8,English,/ebooks/2268,Antony and Cleopatra
9,English,/ebooks/1534,Antony and Cleopatra
10,English,/ebooks/1796,Antony and Cleopatra
11,English,/ebooks/1121,As You Like It
12,English,/ebooks/1523,As You Like It
13,English,/ebooks/1786,As You Like It
